<a href="https://colab.research.google.com/github/artemkush1/ML_predict_CS-GO_matches/blob/master/data_collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pprint as pp
import requests
import datetime
from bs4 import BeautifulSoup
from re import sub
import re
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

pr = pp.PrettyPrinter()
prefix = 'https://www.hltv.org'
max_rank = 1000
last_months = 6

In [ ]:
from sys import argv
import urllib3
from os import system as terminal
import requests
from colorama import Fore,Style

URL = "http://www.hltv.org"
CMD_CLEAR_TERM = "clear"
TIMEOUT = (3.05,27)

def check_proxy(proxy):
    '''
        Function for check proxy return ERROR
        if proxy is Bad else
        Function return None
    '''
    try:
        session = requests.Session()
        session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
        session.max_redirects = 300
        proxy = proxy.split('\n',1)[0]
        print(Fore.LIGHTYELLOW_EX + 'Checking ' + proxy)
        session.get(URL, proxies={'http':'http://' + proxy}, timeout=TIMEOUT,allow_redirects=True)
    except requests.exceptions.ConnectionError as e:
        print(Fore.LIGHTRED_EX + 'Error!')
        return e
    except requests.exceptions.ConnectTimeout as e:
        print(Fore.LIGHTRED_EX + 'Error,Timeout!')
        return e
    except requests.exceptions.HTTPError as e:
        print(Fore.LIGHTRED_EX + 'HTTP ERROR!')
        return e
    except requests.exceptions.Timeout as e:
        print(Fore.LIGHTRED_EX + 'Error! Connection Timeout!')
        return e
    except urllib3.exceptions.ProxySchemeUnknown as e:
        print(Fore.LIGHTRED_EX + 'ERROR unkown Proxy Scheme!')
        return e
    except requests.exceptions.TooManyRedirects as e:
        print(Fore.LIGHTRED_EX + 'ERROR! Too many redirects!')
        return e

proxies = ['82.200.233.4:3128', '41.77.81.90:8080']
goods = 0
terminal(CMD_CLEAR_TERM)
print(Fore.LIGHTCYAN_EX + '===========================================')
for proxy in proxies:
    try:
        if check_proxy(proxy):
            print(Fore.LIGHTRED_EX + 'Bad proxy ' + proxy)
        else:
            print(Fore.LIGHTGREEN_EX + 'Good proxy ' + proxy)
            file_with_goods = open('good.txt','a')
            file_with_goods.write(proxy + '\n')
            goods += 1
        print(Fore.LIGHTCYAN_EX + '=================================================')
    except KeyboardInterrupt:
        print(Fore.LIGHTGREEN_EX + '\nExit.')
        exit()

print(Fore.LIGHTGREEN_EX + 'Total ' + str(goods) + ' good proxies found')
print(Fore.LIGHTRED_EX + 'And ' + str(len(proxies) - goods) + ' is bad')
print()


In [ ]:
import time

proxyDict = None

def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    got_req = requests.get(url, headers=headers, proxies=proxyDict).text
    time.sleep(0.25)
    return BeautifulSoup(got_req, "lxml")

# Functions for each map

In [ ]:


class MatchHolder:
    def __init__(self, match_url):
        self.match_url = match_url
        self.match_page = get_parsed_page(match_url)
        self.team1_url, self.team2_url = self._get_teams_url()
        self.team1_stat_url, self.team2_stat_url = self._get_full_team_stat_page(self.team1_url),\
                                         self._get_full_team_stat_page(self.team2_url)
        self.match_date = datetime.datetime.utcfromtimestamp(int(self._get_unixtime_match()))
        self.maps_played_with_score = self.get_score_on_each_map()
        self.history_h2h = self.get_history_h2h()
        self.team1_page, self.team2_page = get_parsed_page(self.team1_url), get_parsed_page(self.team2_url)
        self.team1_rank, self.team2_rank = self.get_team_rank(self.team1_page), self.get_team_rank(self.team2_page)
        self.event_page = self._get_event_page()
        self.prize_pool = self.get_prize_pool()
        self.count_teams_on_tour = self.get_teams_on_tour()
        self.players_url = self.get_players_from_match_url()
        (self.players_stats_last6m) = self.get_stats_each_player()
         #self.players_stats_last3m,
         #self.players_stats_last9m,
         #self.players_stats_last12m...
        self.maps_team1, self.maps_team2 = self.get_maps_url(self.team1_stat_url), self.get_maps_url(self.team2_stat_url)
        self.stat_maps_team1, self.stat_maps_team2 = self.get_maps_stat_dict(self.maps_team1, 'team1',
                                                            skip_params=['Wins / draws / losses'],
                                                            maps_need=[i for i in self.maps_played_with_score][0]), \
                                                     self.get_maps_stat_dict(self.maps_team2, 'team2',
                                                            skip_params=['Wins / draws / losses'],
                                                            maps_need=[i for i in self.maps_played_with_score[0]])
        self.last5matches = self.get_last5_matches_v1_2teams()
        self.pistol_rounds = self.get_pistols_rounds()
        
    def _get_teams_url(self):
        return [prefix + self.match_page.find_all('div', {"class": "team"})[0].find('a')['href'],
                prefix + self.match_page.find_all('div', {"class": "team"})[1].find('a')['href']]
    
    def _get_full_team_stat_page(self, team_url):
        return f'https://www.hltv.org/stats/teams/maps{team_url.split("https://www.hltv.org/team")[-1]}'
    
    def _get_unixtime_match(self):
        return self.match_page.find('div', {"class": "timeAndEvent"}).find('div', {"class": "time"})['data-unix'][:-3]
    
    def _get_event_page(self):
        return get_parsed_page(prefix + self.match_page.find('div', {'class': 'event'}).find('a')['href'])
    
    # return {maps: score on map}, total-maps-counter
    def get_score_on_each_map(self):
        def _get_maps_():
            maps_score = []
            maps_obj = self.match_page.find('div', {"class": "flexbox-column"}).find_all('div', {"class": "mapholder"})
            for i in maps_obj:
                tmp = i.find('div', {"class": "played"})
                if tmp:
                    maps_score += [i]
            return maps_score

        maps_obj = _get_maps_()
        # print(len(maps_obj))
        maps = {}
        i = 0
        for map_ in maps_obj:
            tmp1 = int(map_.find('div', {"class": "results-left"}).find('div', {"class": "results-team-score"}).text)
            tmp2 = int(map_.find('span', {"class": "results-right"}).find('div', {"class": "results-team-score"}).text)
            if len([tmp1, tmp2]) != 2:
                print("Problems in get_score()")
            maps[map_.find('div', {"class": "mapname"}).text] = [tmp1, tmp2]
            i += 1
        return maps, i
    
    # get history_h2h: expect 3 params
    def get_history_h2h(self):
        tmp = [int(i.find('div', {'class': 'bold'}).text) for i in
               self.match_page.find_all('div', {'class': 'flexbox-center'})]
        if len(tmp) != 3:
            print("Problems in get_history_h2h()")
        return tmp
    
    # get teams_rank: expect 1 param
    def get_team_rank(self, team):
        try:
            tmp = int(team.find('div', {'class': 'profile-team-stats-container'}).
                                 find_all('span', {'class': "right"})[0].text[1:])
            if not tmp:
                return max_rank
            return tmp
        except:
            return max_rank
    
    # get prize_pool: expect 1 param
    def get_prize_pool(self):
        try:
            tmp = int(sub(r'[^\d.]', '', self.event_page.find('td', {'class': 'prizepool'}).text))
            if not tmp:
                return 0
            return tmp
        except:
            return 0
        
    # get teams_on_tour: expect 1 param
    def get_teams_on_tour(self):
        tmp = int(re.findall(r"[-+]?\d*\.\d+|\d+", self.event_page.find('td', {'class': 'teamsNumber'}).text)[0])
        if tmp:
            return tmp
        return 0
    
    # expect 2 * 5 params
    def get_players_from_match_url(self):
        """tmp = [[prefix + '/stats/players' + (i.find('a')['href'].split('/player')[-1])
                 for i in k.find_all('td', {'class': 'player'})][:5]
                for k in self.match_page.find_all('div', {'class': 'lineup standard-box'})]"""
        tmp = {}
        all_pl = self.match_page.find_all('div', {'class': 'lineup standard-box'})
        for k in range(2):
            for i in range(5):
                try:
                    tmp[f'team{k}_pl{i}'] = prefix + '/stats/players' + \
                            (all_pl[k].find_all('td', {'class': 'player'})[i].find('a')['href'].split('/player')[-1])
                except:
                    tmp[f'team{k}_pl{i}'] = ""
        return tmp

    # expect 19 params in dict
    def _get_player_many_stats(self, player, index):
        table_up = player.find('div', {'class': 'statistics'}).find_all('div', {'class': 'stats-row'})
        res = {}
        for i in table_up:
            tmp1 = i.find_all('span')[-1].text
            if tmp1[-1] == '%':
                tmp1 = float(tmp1[:-1]) / 100
            elif tmp1[-1] == '-':
                tmp1 = '0'
            res[i.find_all('span')[0].text + index] = tmp1
        table_down = player.find('div', {'class': 'featured-ratings-container'}).find_all('div', {'class': 'col-custom'})
        for i in table_down:
            tmp1 = i.find('div', {'class': 'rating-value'}).text
            if tmp1[-1] == '%':
                tmp1 = float(tmp1[:-1]) / 100
            elif tmp1[-1] == '-':
                tmp1 = '0'
            res[i.find('div', {'class': 'rating-description'}).text + index] = tmp1
        if len(res) != 19:
            print("Problems in get_player_many_stats()")
        return res

    # return dict with 19 params
    def _get_player_stat_in_period(self, player_url, date_start, date_end, index):
        player_page = get_parsed_page(player_url + \
                      f'?startDate={date_start.strftime("%Y-%m-%d")}&endDate={date_end.strftime("%Y-%m-%d")}')
        return self._get_player_many_stats(player_page, index)
    
    # return 5 big dict: in each 5 players with 19 params
    def get_stats_each_player(self):
        #_1m = {}
        #_3m = {}
        _6m = {}
        #_9m = {}
        #_12m = {}
        for dict_, months in zip([_6m],
                     [6]):
            for i in self.players_url:
                # print(i)
                try:
                    dict_[i] = [self._get_player_stat_in_period(self.players_url[i],
                                                                self.match_date - relativedelta(months=months),
                                                                self.match_date - relativedelta(days=1),
                                                                '_'+i)]
                except Ex:
                    dict_[i] = [0 for t in range(19)]
        return _6m  # , _6m, _9m, _12m

    # get url on maps
    def get_maps_url(self, team_url_maps):
        dict_ = {}
        # print(team_url_maps)
        maps = get_parsed_page(team_url_maps).find('div', {'class': 'two-grid'}).find_all('div', {'class': 'map-pool'})
        for i in maps:
            dict_[i.find('img', {'class': 'map-pool-map'})['title']] = prefix + i.find('a')['href']
        return dict_
    
    # expect (10 - skip_params) parameters in dict
    def _get_map_many_stats(self, map_page, index, skip_params):
        table = map_page.find('div', {'class': 'stats-rows'}).find_all('div', {'class': 'stats-row'})
        stats = {}
        for k in table:
            if k.find_all('span')[0].text not in skip_params:
                tmp = k.find_all('span')[-1].text
                if tmp[-1] == '%':
                    tmp = float(tmp[:-1]) / 100
                elif tmp[-1] == '-':
                    tmp = 0
                stats[k.find_all('span')[0].text + index] = float(tmp)
        if len(stats) != (10 - len(skip_params)):
            print("Problems in get_stats_on_each_map() (10 params)")
        return stats
    
    # expect dict
    def _get_maps_stat_in_period(self, match_url, date_start, date_end, index, skip_params):
        map_page = get_parsed_page(match_url + \
                      f'?startDate={date_start.strftime("%Y-%m-%d")}&endDate={date_end.strftime("%Y-%m-%d")}')
        return self._get_map_many_stats(map_page, index, skip_params)

    # expect 5 big DICTS with (10 - skip_params) parameters in each map
    def get_maps_stat_dict(self, maps, index, skip_params=[], maps_need=None):
        # self.maps_team1, self.maps_team2
        #_1m = {}
        #_3m = {}
        _6m = {}
        #_9m = {}
        #_12m = {}
        for dict_, months in zip([_6m],
                     [6]):
            for i in maps:
                # print(i)
                if i in maps_need:
                    dict_[i] = [self._get_maps_stat_in_period(maps[i],
                               self.match_date - relativedelta(months=months),
                               self.match_date - relativedelta(days=1),
                               '_'+index, skip_params)]
                elif maps_need is None:
                    dict_[i] = [self._get_maps_stat_in_period(maps[i],
                               self.match_date - relativedelta(months=months),
                               self.match_date - relativedelta(days=1),
                               '_'+index, skip_params)]
        return [_6m]


    # get lats 5 matches for both teams: expect 5 params in 2 dicts in 2 other dicts
    def get_last5_matches_v1_2teams(self):
        def get_url_opponent(url1, i):
            return prefix + url1.find_all('tr', {'class': 'table'})[i].find('a', {'class': 'text-ellipsis'})['href']

        def get_type_match(url1, i):
            return url1.find_all('tr', {'class': 'table'})[i].find('a', {'data-link-tracking-page': 'Matchpage'}).text

        def get_score_array(url1, i):
            return [int(k) for k in
                    url1.find_all('tr', {'class': 'table'})[i].find('td', {'class': 'spoiler'}).text.split(' - ')]

        def get_all(table):
            team_ranks = {}
            team_scores = {}
            for i in range(5):
                try:
                    rank_ = self.get_team_rank(get_parsed_page(get_url_opponent(table, i)))
                    score1_, score2_ = get_score_array(table, i)
                    team_ranks[f'opponent_rank_from_5_last_matches_{i}'] = rank_
                    # print('scores last 5 matches:', score1_, score2_)
                    if score1_ > score2_:
                        team_scores[f'win/loss_from_5_last_matches_{i}'] = 1
                    else:
                        team_scores[f'win/loss_from_5_last_matches_{i}'] = -1
                except Exception as e:
                    print('Problems in get_last5_matches_v1_2teams()', e)
                    team_ranks[f'opponent_rank_from_5_last_matches_{i}'] = '0'
                    team_scores[f'win/loss_from_5_last_matches_{i}'] = 0
            return team_ranks, team_scores
        last5m = {}
        tables = (self.match_page.find('div', {'class': 'past-matches'}).
                 find_all('div', {'class': 'half-width'}))
        last5m['team1'] = [get_all(tables[0])]
        last5m['team2'] = [get_all(tables[1])]
        # last5_matches_2teams = [*types, *teams_rank, *scores]
        if len(last5m) != 2:
            print('Problems in get_last5_matches_v1_2teams()')
        return last5m

    def get_pistols_rounds(self):
        def _get_maps_():
            maps_score = []
            maps_obj = self.match_page.find('div', {"class": "flexbox-column"}).find_all('div', {"class": "mapholder"})
            for i in maps_obj:
                tmp = i.find('div', {"class": "played"})
                if tmp:
                    maps_score += [i]
            return maps_score

        maps_obj = _get_maps_()
        # print(len(maps_obj))
        maps = {}
        i = 0
        for map_ in maps_obj:
            tmp = prefix + map_.find('div', {"class": "results-center"}).\
                                find('a', {"class": "results-stats"})['href']
            page_stat = get_parsed_page(tmp)
            tables = page_stat.find_all('div', {"class": "round-history-half"})[:2]
            pistols = {}
            n = 1
            for k in tables:
                if k.find('img', {"class": "round-history-outcome"})['src'] !=\
                            '//static.hltv.org/images/scoreboard/emptyHistory.svg':
                    pistols[f'pistol_round_{n}'] = 1
                else:
                    pistols[f'pistol_round_{n}'] = -1
                n += 14
            maps[map_.find('div', {"class": "mapname"}).text] = [pistols]
        return maps



# get_metches_url(filename='matches.txt', pages_with_results=[0, 1])
# cl = MatchHolder('https://www.hltv.org/matches/2341715/vitality-vs-dignitas-blast-premier-spring-2020-europe-showdown')
# print(cl.stat_maps_team1)
# cl.pistol_rounds


In [ ]:
# download all urls matches
def get_matches_url(filename=None, pages_with_results=[0]):
    all_matches = []
    for i in pages_with_results:
        if i == 0:
            results = get_parsed_page('https://www.hltv.org/results')
        else:
            results = get_parsed_page(f'https://www.hltv.org/results?offset={i}00')
        # list of matches
        all_matches += [prefix + url_.find('a', {"class": "a-reset"})['href'] for url_ in
                        results.find('div',
                                     {"class": "results-all", 'data-zonedgrouping-group-classes': "results-sublist"}).
                        find_all('div', {"class": "result-con"})]
    all_matches = np.array(all_matches)
    # print(all_matches)
    if filename is None:
        return all_matches
    else:
        np.save(filename, np.array(all_matches))


#get_matches_url(filename='all_matches', pages_with_results=[i for i in range(100)])

In [ ]:
# cl = MatchHolder('https://www.hltv.org/matches/2341715/vitality-vs-dignitas-blast-premier-spring-2020-europe-showdown')

In [ ]:
def get_pist_r(stat, name):
    res = []
    lb = []
    for i in stat[name][0]:
        res += [stat[name][0][i]]
        lb += [i]
    return res, lb
    


def get_big_stat_map(stat, index, name):
    res = []
    lb = []
    for i in stat[index][name][0]:
        res += [stat[index][name][0][i]]
        lb += [i]
    return res, lb

def get_big_player_stat(stat):
    labels = []
    res = []
    for player in stat:
        for k in stat[player][0]:
            labels += [k]
            res += [stat[player][0][k]]
    return res, labels

def get_last_5m(stat):
    labels = []
    res = []
    for i in stat:
        for k in stat[i][0]:
            for j in k:
                labels += [j]
                res += [k[j]]
    return res, labels

def prepare_maps_played(stat):
    dict_ = {}
    for i in stat:
        dict_[i] = ['score1', 'score2']
    return stat, dict_


In [ ]:


FINAL_DICT = {0: None}


def final_def(cl):
    # Взял статистику по всем картам и количество сыгранных карт
    maps_played, maps_total = cl.maps_played_with_score  # added
    maps_played_score, lab_maps_played = prepare_maps_played(maps_played)  # added

    ranks_score_last5, labels_last5 = get_last_5m(cl.last5matches)  # added

    # Рассматриваем одну из карт
    for name in maps_played_score:
        score = maps_played_score[name]  # added
        lbscore = lab_maps_played[name]  # added
        # Сохраняем данные за разный период
        for i, big_data_map in zip([0], [cl.players_stats_last6m]): ## !!!!!!
            player_stat, lab_pl_stat = get_big_player_stat(big_data_map)  # OK  # added
            maps_stat1, lab_map_stat1 = get_big_stat_map(cl.stat_maps_team1, i, name)  # OK  # added
            maps_stat2, lab_map_stat2 = get_big_stat_map(cl.stat_maps_team2, i, name)  # OK  # added
            pist, lbpist = get_pist_r(cl.pistol_rounds, name)

            final_data = np.array\
            ([cl.match_url, *score, maps_total, *pist,
             cl.team1_rank, cl.team2_rank,
             cl.prize_pool, cl.count_teams_on_tour, 
             *ranks_score_last5, *player_stat, *maps_stat1, *maps_stat2
            ])

            labels = np.array\
            (['url', *lbscore, 'maps_total', *lbpist,
             'team1_rank', 'team2_rank',
             'prize_pool', 'teams_on_tour',
             *labels_last5, *lab_pl_stat, *lab_map_stat1, *lab_map_stat2
            ])
            if FINAL_DICT[i] is None:
                FINAL_DICT[i] = pd.DataFrame([final_data],
                                             columns=labels)
            else:
                FINAL_DICT[i] = pd.concat([FINAL_DICT[i],
                                           pd.DataFrame([final_data],
                                                        columns=labels)])


In [14]:
import time


matches = np.load('all_matches.npy', allow_pickle=True)


for i in FINAL_DICT:
    FINAL_DICT[0] = pd.read_csv('df_0.csv').drop(columns=['Unnamed: 0'])


counter = 0
for match in matches:
    print(counter, match)
    try:
        cl = MatchHolder(match)
        final_def(cl)
        counter += 1
    except Exception as e:
        print(e)
    for i in FINAL_DICT:
        FINAL_DICT[i].to_csv(f'df_{i}.csv')



Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Number of manager items must equal union of block items
# manager items: 231, # tot_items: 241
7678 https://www.hltv.org/matches/2332151/forze-vs-heroic-ggbet-sydney-invitational
7679 https://www.hltv.org/matches/2332041/gamerlegion-vs-valiance-united-masters-league-finals
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_ma

7707 https://www.hltv.org/matches/2332101/forze-vs-giants-kalashnikov-cup-season-2
7708 https://www.hltv.org/matches/2332102/nemiga-vs-unicorns-of-love-kalashnikov-cup-season-2
7709 https://www.hltv.org/matches/2332125/devilsone-vs-forze-ggbet-sydney-invitational
7710 https://www.hltv.org/matches/2332100/owms-vs-dreameaters-kalashnikov-cup-season-2
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
7711 https://www.hltv.org/matches/2331947/paradox-vs-breakaway-esl-australia-nz-championship-season-8
7712 https://www.hltv.org/matches/2332129/order-vs-grayhound-esea-mdl-season-30-australia
7713 https://www.hltv.org/matches/2331946/fighting-4-freedom-vs-chiefs-esl-australia-nz-championship-season-8
Problems in get_l

7744 https://www.hltv.org/matches/2331811/intact-vs-singularity-esea-mdl-season-30-north-america
7745 https://www.hltv.org/matches/2331870/the-quest-vs-identitydisorder-esea-advanced-season-30-north-america
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Number of manager items must equal union of block items
# manager items: 232, # tot_items: 242
7745 https://www.hltv.org/matches/2331876/zone-vs-we-dont-give-a-hoot-esea-advanced-season-30-north-america
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems

Number of manager items must equal union of block items
# manager items: 236, # tot_items: 246
7786 https://www.hltv.org/matches/2332000/w7m-vs-imperial-dreamhack-masters-dallas-2019-south-america-closed-qualifier
7787 https://www.hltv.org/matches/2331599/astralis-vs-liquid-blast-pro-series-so-paulo-2019
7788 https://www.hltv.org/matches/2331976/eunited-vs-furia-dreamhack-masters-dallas-2019-north-america-closed-qualifier
7789 https://www.hltv.org/matches/2331983/complexity-vs-envy-dreamhack-masters-dallas-2019-north-america-closed-qualifier
7790 https://www.hltv.org/matches/2331985/sharks-vs-isurus-dreamhack-masters-dallas-2019-south-america-closed-qualifier
7791 https://www.hltv.org/matches/2331984/denial-vs-luminosity-dreamhack-masters-dallas-2019-north-america-closed-qualifier
7792 https://www.hltv.org/matches/2331598/ence-vs-mibr-blast-pro-series-so-paulo-2019
'NoneType' object has no attribute 'find'
7792 https://www.hltv.org/matches/2331997/tricked-vs-codewise-unicorns-good-game

7827 https://www.hltv.org/matches/2331930/absolute-vs-d13-iem-sydney-2019-east-asia-closed-qualifier
7828 https://www.hltv.org/matches/2331924/beyond-vs-boot-ds-iem-sydney-2019-southeast-asia-closed-qualifier
7829 https://www.hltv.org/matches/2331970/ghost-vs-envy-dreamhack-masters-dallas-2019-north-america-closed-qualifier
7830 https://www.hltv.org/matches/2331974/singularity-vs-luminosity-dreamhack-masters-dallas-2019-north-america-closed-qualifier
7831 https://www.hltv.org/matches/2331841/ex-bragantino-vs-all-knights-esea-open-season-30-brazil
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
7832 https://www.hltv.org/matches/2331972/detona-vs-weplaygames-dreamhack-masters-dallas-2019-south-america-closed-qualifier
7833 https://www.hltv.org/matches/2331971/evidence-vs-furi

...

Problems in get_last5_matches_v1_2teams() list index out of range
Number of manager items must equal union of block items
# manager items: 234, # tot_items: 244
7999 https://www.hltv.org/matches/2331618/carnage-fe-vs-pain-fe-wesg-2018-world-finals-female
Problems in get_last5_matches_v1_2teams() list index out of range
8000 https://www.hltv.org/matches/2331664/g2-vs-panda-wesg-2018-world-finals
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
8001 https://www.hltv.org/matches/2331663/absolute-vs-tnc-wesg-2018-world-finals
8002 https://www.hltv.org/matches/2331616/lazarus-fe-vs-france-fe-wesg-2018-world-finals-female
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_last5_matches_v1_2teams()

8020 https://www.hltv.org/matches/2331712/alternate-attax-vs-ex-fragsters-esea-mdl-season-30-europe
8021 https://www.hltv.org/matches/2331710/expert-vs-spirit-esea-mdl-season-30-europe
8022 https://www.hltv.org/matches/2331702/devilsone-vs-volgare-noxfire-league-season-2
8023 https://www.hltv.org/matches/2331732/ence-vs-uruguay-esea-mdl-season-30-europe
8024 https://www.hltv.org/matches/2331730/2019-vs-giants-charleroi-esports-europe-qualifier-1
8025 https://www.hltv.org/matches/2331709/vitality-vs-unicorns-of-love-esea-mdl-season-30-europe
8026 https://www.hltv.org/matches/2331731/sprout-vs-fastlose-charleroi-esports-europe-qualifier-1
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_last5_matches_v1_2teams() list index out of range
'NoneType' object is not subscriptable
8026 https://www.hltv.org/matches/2331708/alternate-attax-vs-vitality-esea-mdl-season-30-europe
8027 https://www.hltv.org/matches/2331706/chaos-vs-ence-esea-mdl-season-30-europe


Problems in get_player_many_stats()
Problems in get_player_many_stats()
Number of manager items must equal union of block items
# manager items: 230, # tot_items: 240
8072 https://www.hltv.org/matches/2330914/vitality-vs-ex-fragsters-esea-mdl-season-30-europe
'NoneType' object has no attribute 'find'
8072 https://www.hltv.org/matches/2331395/g2-vs-revolution-wesg-2018-world-finals
Problems in get_player_many_stats()
Number of manager items must equal union of block items
# manager items: 229, # tot_items: 239
8072 https://www.hltv.org/matches/2331400/valiance-vs-optic-wesg-2018-world-finals
8073 https://www.hltv.org/matches/2331407/furious-vs-absolute-wesg-2018-world-finals
8074 https://www.hltv.org/matches/2331401/denial-vs-nasr-wesg-2018-world-finals
8075 https://www.hltv.org/matches/2331534/off-the-grid-vs-grayhound-esea-mdl-season-30-australia
8076 https://www.hltv.org/matches/2331398/valiance-vs-denial-wesg-2018-world-finals
8077 https://www.hltv.org/matches/2331462/breakaway-vs-g

Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Problems in get_last5_matches_v1_2teams() list index out of range
Number of manager items must equal union of block items
# manager items: 237, # tot_items: 247
8099 https://www.hltv.org/matches/2331628/shipachi-vs-zorka-meta-19
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Problems in get_last5_matches_v1_2teams() list index out of range
Problems 

8120 https://www.hltv.org/matches/2331547/gamepub-vs-devilsone-noxfire-league-season-2
Problems in get_player_many_stats()
Problems in get_player_many_stats()
Number of manager items must equal union of block items
# manager items: 230, # tot_items: 240
8120 https://www.hltv.org/matches/2331564/windigo-vs-nemiga-ecs-season-7-europe-pinnacle-cup
8121 https://www.hltv.org/matches/2331546/unicorns-of-love-vs-nexus-noxfire-league-season-2
8122 https://www.hltv.org/matches/2331566/movistar-riders-vs-chaos-ecs-season-7-europe-pinnacle-cup
8123 https://www.hltv.org/matches/2331551/expert-vs-uruguay-esea-mdl-season-30-europe
8124 https://www.hltv.org/matches/2331250/izako-boars-vs-virtuspro-esea-mdl-season-30-europe
8125 https://www.hltv.org/matches/2331545/3dmax-vs-devilsone-noxfire-league-season-2
8126 https://www.hltv.org/matches/2331550/alternate-attax-vs-unicorns-of-love-esea-mdl-season-30-europe
8127 https://www.hltv.org/matches/2331548/epsilon-vs-forze-esea-mdl-season-30-europe
8128 htt